# Mount Baker CSM stereo run (with core 128 km^2 dataset)
- Shashank Bhushan and Seth Vanderwilt
- May 2023

- Demonstration of the workflow to create a stack of inputs for elevation refinement models- this notebook shows execution with Mount Baker 20150911 stereo pair based on experiments with May 2019 South Cascade Glacier stereo processing.
- Remaining issue
  * Alignment with bare ground lidar brings stereo rasters close to lidar DEM but they do not align perfectly. Investigate and fix - previous Baker processing script (`pc_laz_prep_full.sh`) resulted in lidar DEM and stereo DEM lining up perfectly.
- Next steps:
  * Remove remaining hardcoding of filenames / ASP outputs / CRS
  * Set reasonable bounds based on stereo pair & lidar intersection
  * Pull these revised commands back into generic stereo script rather than having to execute in notebook
  * (In script) final steps of copying stack of desired rasters into single folder

In [13]:
pair_dir = "/mnt/1.0_TB_VOLUME/sethv/shashank_data/WV01_20150911_1020010042D39D00_1020010043455300"


## Find the left and right images based on XML metadata

In [14]:
import os
import xml.etree.ElementTree as ET
import glob
import rioxarray

In [31]:
xml_pattern = os.path.join(pair_dir, "*.xml")
image_gsds = []
for xml_fn in glob.glob(xml_pattern):
    root = ET.parse(xml_fn)
    # Extract mean GSD from the metadata fields
    meanGSD = round(float(root.find("./IMD/IMAGE/MEANPRODUCTGSD").text),3)

    print(os.path.basename(xml_fn), meanGSD)
    image_id = os.path.basename(xml_fn).split(".")[0]
    
    # Extract fields of interest and just keep all metadata around as 'xml_root'
    image_dict = {
        "xml_fn": xml_fn,
        "image_fn": xml_fn.replace("xml", "tif"),
        "meanGSD": meanGSD,
        "image_id": image_id,
        "xml_root": root
    }
    
    image_gsds.append(image_dict)
    print(image_id)

left_dict = min(image_gsds, key=lambda i: i["meanGSD"])
right_dict = max(image_gsds, key=lambda i: i["meanGSD"])
res = min(image_gsds, key=lambda i: i["meanGSD"])["meanGSD"]
print(f"Highest resolution to use for orthoimages & stereo: {res} meters")

left_image_id = left_dict["image_id"]
right_image_id = right_dict["image_id"]

left_image_fn = left_dict["image_fn"]
right_image_fn = right_dict["image_fn"]

1020010043455300.r100.xml 0.6
1020010043455300
1020010042D39D00.r100.xml 0.541
1020010042D39D00
Highest resolution to use for orthoimages & stereo: 0.541 meters


## TODO: Determine common intersection extent
* Better way: extract URLON ULLAT etc. fields from the XML and find the shared bbox between the images and lidar point cloud to use for rest of dataset creation.

In [16]:
xmin, ymin, xmax, ymax = (583000, 5395000, 591000, 5411000)  # bounds in reprojected CRS


## Bundle adjustment

In [ ]:
!bundle_adjust --ip-per-image 10000 -t dg --dg-use-csm \
    --camera-weight 0 --tri-weight 0.1 --tri-robust-threshold 0.1 \
    $left_image_fn $right_image_fn $left_image_xml $right_image_xml \
    -o dg_csm_model_refined/run

## Register camera models to reference DEM
* Using Copernicus GLO-30 Digital Elevation Model from [OpenTopography](https://portal.opentopography.org/raster?opentopoID=OTSDEM.032021.4326.3)
* Coordinate System:
  - Horizontal: WGS84 [EPSG: 4326]
  - Vertical: EGM2008 [EPSG: 3855] 

In [ ]:
refdem_from_opentopography = "output_COP30.tif"
refdem = "refdem_copernicus_30m_32610_wgs84.tif" # reprojected & cropped to extent

## Adjust reference DEM to desired datum.
* Call `dem_geoid` - but only in newer ASP versions with bugfix for .jp2 datum files

In [ ]:
# Adjust the Copernicus 30m product to be in heights above ellipsoid instead of EGM2008 geoid
# dem_geoid bug fixed in ASP 3.2.1+
!/mnt/1.0_TB_VOLUME/sethv/dshean_tools/StereoPipeline-3.2.1-alpha-2023-05-11-x86_64-Linux/bin/dem_geoid --reverse-adjustment --geoid EGM2008 output_COP30.tif


In [ ]:
refdem_from_opentopography_ellipsoid = refdem_from_opentopography.replace(".tif", "-adj.tif")
!gdalwarp -overwrite -r cubic -t_srs "EPSG:32610" -te $xmin $ymin $xmax $ymax $refdem_from_opentopography_ellipsoid $refdem


In [17]:
t_srs = "EPSG:32610"  # add vertical CRS if compatible with ASP?

In [ ]:
refdem_rxr = rioxarray.open_rasterio(refdem)
refdem_opentopography_rxr = rioxarray.open_rasterio(refdem_from_opentopography)
diff = refdem_rxr - refdem_opentopography_rxr.rio.reproject_match(refdem_rxr)
print(f"Refdem in EGM2008 heights minimum elevation: {refdem_opentopography_rxr.min().item()}")
print(f"Adjusted refdem minimum elevation: {refdem_rxr.min().item()}")


**Plot the difference after vertical datum adjustment - pixel shape artifacts expected since adjusted refdem is interpolated and raw refdem is not.**

In [ ]:
import matplotlib.pyplot as plt

diff.plot(vmin=-30, vmax=30, cmap="RdBu")
plt.title(f"Mean difference after datum change (EGM2008 -> WGS84): {diff.mean().item():.2f} m")


## Align sparse point cloud (which comes from initial bundle adjustment) to Copernicus DEM
* Note: this should not result in much vertical translation, but some horizontal translation

In [ ]:
sparse_cloud_fn = "dg_csm_model_refined/run-final_residuals_pointmap.csv"

In [ ]:
# TODO: should this alignment be translation only, or continue to allow rotation?
!pc_align --max-displacement 20 --csv-format "2:lat 1:lon 3:height_above_datum" --save-transformed-source-points $refdem $sparse_cloud_fn -o align_sparse_ba_cloud_to_cop30/align_sparse_ba_cloud_to_cop30 


## Update camera models for mapproject

In [ ]:
left_adjust_fn = f"dg_csm_model_refined/run-{left_image_id}.r100.adjusted_state.json"
right_adjust_fn = f"dg_csm_model_refined/run-{right_image_id}.r100.adjusted_state.json"
!bundle_adjust -t csm --apply-initial-transform-only yes $left_image_fn $right_image_fn \
    $left_adjust_fn $right_adjust_fn \
    --initial-transform align_sparse_ba_cloud_to_cop30/align_sparse_ba_cloud_to_cop30-transform.txt \
    -o csm_cameras_aligned_to_copernicus/csm_cameras_aligned_to_copernicus

## Orthorectification

In [ ]:
# lowres = 1.0 # meters
# left_orthoimage_lowres_fn = f"ortho_left_{lowres}.tif"
# !mapproject -t csm --ot UInt16 --t_srs $t_srs --t_projwin $xmin $ymin $xmax $ymax \
#      --tr $lowres \
#     $refdem \
#     $left_image_fn $left_adjust_fn $left_orthoimage_lowres_fn

In [ ]:
left_bundle_adjusted_and_aligned_csm_fn = f"csm_cameras_aligned_to_copernicus/csm_cameras_aligned_to_copernicus-run-{left_image_id}.r100.adjusted_state.json"
right_bundle_adjusted_and_aligned_csm_fn = f"csm_cameras_aligned_to_copernicus/csm_cameras_aligned_to_copernicus-run-{right_image_id}.r100.adjusted_state.json"


## Mapprojection at full resolution

In [ ]:
left_orthoimage_fullres_fn = f"ortho_left_{res}m.tif"
!mapproject -t csm --ot UInt16 --t_srs $t_srs \
     --t_projwin $xmin $ymin $xmax $ymax \
     --tr $res \
    $refdem \
    $left_image_fn $left_bundle_adjusted_and_aligned_csm_fn $left_orthoimage_fullres_fn

In [ ]:
right_orthoimage_fullres_fn = f"ortho_right_{res}m.tif"
!mapproject -t csm --ot UInt16 --t_srs $t_srs \
     --t_projwin $xmin $ymin $xmax $ymax \
     --tr $res \
    $refdem \
    $right_image_fn $right_bundle_adjusted_and_aligned_csm_fn $right_orthoimage_fullres_fn

## Best parallel_stereo command for run 2: same parameters, correctly reprojected and interpolated refdem
* Explanations of stereo parameters go in the README

In [ ]:
# Running this in Tmux
!parallel_stereo \
    --corr-kernel 7 7 \
    --cost-mode 3  \
    --subpixel-kernel 15 15 \
    --subpixel-mode 9 \
    --stereo-algorithm asp_mgm \
    --alignment-method none \
    --num-matches-from-disparity 10000 \
    --corr-tile-size 1024 \
    --corr-memory-limit-mb 5000 \
    --erode-max-size 0 \
    ortho_left_0.541m.tif \
    ortho_right_0.541m.tif \
    csm_cameras_aligned_to_copernicus/csm_cameras_aligned_to_copernicus-run-1020010042D39D00.r100.adjusted_state.json \
    csm_cameras_aligned_to_copernicus/csm_cameras_aligned_to_copernicus-run-1020010043455300.r100.adjusted_state.json \
    stereo_processing/run_large_2 \
    refdem_copernicus_30m_32610_wgs84.tif

## DEM quality check before coregistration: create DEMs at different grid sizes

In [ ]:
# Can also try 2 or 3 meter grid and resample for smoother results if needed?
# !point2dem --tr 2.0 stereo_processing/run_large_2-PC.tif -o "stereo_processing/run_large_2-2.0m"
!point2dem --tr 1.0 --errorimage stereo_processing/run_large_2-PC.tif -o "stereo_processing/run_large_2-1.0m"

## Align stereo point cloud to bare-ground filtered lidar point cloud

In [ ]:
lidar_reference_bareground_laz="/mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/ResDepth/torchgeo_experiments/usgs_all616_laz_filtered_dem_mask_nlcd_rock_exclude_glaciers/merged/merged_baker_bareground_all.laz"


### Update: because the first pc_align call introduced a slight rotation, need to run allowing rotation

In [ ]:
# Run in shell/script, very slow for notebook use
# Removed translation-only argument!
!pc_align --max-displacement 20 --save-transformed-source-points \
$lidar_reference_bareground_laz \
stereo_processing/run_large_2-PC.tif \
-o stereo_processing/translation_alignment/run  \
--highest-accuracy


In [ ]:
# TODO split out & fix exact extent
aligned_stereo_pc = "stereo_processing/translation_alignment/run-trans_source.tif"
aligned_stereo_dem_prefix = "stereo_processing/translation_alignment/aligned_stereo_1.0m"
aligned_stereo_dem_fn = aligned_stereo_dem_prefix + "-DEM.tif"
!point2dem --tr 1 --t_srs $t_srs --errorimage --nodata-value -9999 \
$aligned_stereo_pc -o stereo_processing/translation_alignment/aligned_stereo_1.0m

## Align self-consistent CSM camera models to lidar


In [ ]:
!bundle_adjust -t csm --apply-initial-transform-only yes \
    $left_image_fn \
    $right_image_fn \
    "$left_bundle_adjusted_and_aligned_csm_fn" \
    "$right_bundle_adjusted_and_aligned_csm_fn" \
    --initial-transform try_pc_align_to_lidar_15m_maxdisp_rotationallowed/run-transform.txt \
    -o stereo_processing/translation_alignment/lidar_aligned_csm_cameras

In [25]:
output_res = 1.0


In [20]:
left_final_aligned_csm_fn = f"stereo_processing/translation_alignment/lidar_aligned_csm_cameras-csm_cameras_aligned_to_copernicus-run-{left_image_id}.r100.adjusted_state.json"
right_final_aligned_csm_fn = f"stereo_processing/translation_alignment/lidar_aligned_csm_cameras-csm_cameras_aligned_to_copernicus-run-{right_image_id}.r100.adjusted_state.json"


# Redefine aligned_stereo_dem_fn


### Redo the mapprojection

In [21]:
aligned_stereo_dem_fn = "try_pc_align_to_lidar_15m_maxdisp_rotationallowed/try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-DEM.tif"

In [36]:
aligned_stereo_dem_fn_filled = "filled_dems/test_gdalfill_si3.tif"
#"baker_csm_stack/try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-DEM_holes_filled.tif"

In [37]:
right_orthoimage_fullres_fn = f"final_ortho_right_{output_res:.1f}m.tif"

!mapproject -t csm --ot UInt16 --t_srs $t_srs \
 --t_projwin $xmin $ymin $xmax $ymax \
 --tr $output_res \
$aligned_stereo_dem_fn_filled $right_image_fn $right_final_aligned_csm_fn $right_orthoimage_fullres_fn

mapproject_single --query-projection filled_dems/test_gdalfill_si3.tif /mnt/1.0_TB_VOLUME/sethv/shashank_data/WV01_20150911_1020010042D39D00_1020010043455300/1020010043455300.r100.tif stereo_processing/translation_alignment/lidar_aligned_csm_cameras-csm_cameras_aligned_to_copernicus-run-1020010043455300.r100.adjusted_state.json final_ortho_right_1.0m.tif -t csm --ot UInt16 --t_srs EPSG:32610 --t_projwin 583000 5395000 591000 5411000 --tr 1.0
	--> Setting number of processing threads to: 4
Using session: csm
Loading camera model: /mnt/1.0_TB_VOLUME/sethv/shashank_data/WV01_20150911_1020010042D39D00_1020010043455300/1020010043455300.r100.tif stereo_processing/translation_alignment/lidar_aligned_csm_cameras-csm_cameras_aligned_to_copernicus-run-1020010043455300.r100.adjusted_state.json
Output pixel size: 1
Projected space bounding box: (Origin: (583000, 5395001) width: 7999 height: 15999)
Image box: (Origin: (0, 0) width: 8000 height: 16000)
Output image size:
(width: 8000 height: 16000)


In [38]:
left_orthoimage_fullres_fn = f"final_ortho_left_{output_res:.1f}m.tif"

!mapproject -t csm --ot UInt16 --t_srs $t_srs \
 --t_projwin $xmin $ymin $xmax $ymax \
 --tr $output_res \
$aligned_stereo_dem_fn_filled $left_image_fn $left_final_aligned_csm_fn $left_orthoimage_fullres_fn

mapproject_single --query-projection filled_dems/test_gdalfill_si3.tif /mnt/1.0_TB_VOLUME/sethv/shashank_data/WV01_20150911_1020010042D39D00_1020010043455300/1020010042D39D00.r100.tif stereo_processing/translation_alignment/lidar_aligned_csm_cameras-csm_cameras_aligned_to_copernicus-run-1020010042D39D00.r100.adjusted_state.json final_ortho_left_1.0m.tif -t csm --ot UInt16 --t_srs EPSG:32610 --t_projwin 583000 5395000 591000 5411000 --tr 1.0
	--> Setting number of processing threads to: 4
Using session: csm
Loading camera model: /mnt/1.0_TB_VOLUME/sethv/shashank_data/WV01_20150911_1020010042D39D00_1020010043455300/1020010042D39D00.r100.tif stereo_processing/translation_alignment/lidar_aligned_csm_cameras-csm_cameras_aligned_to_copernicus-run-1020010042D39D00.r100.adjusted_state.json
Output pixel size: 1
Projected space bounding box: (Origin: (583000, 5395001) width: 7999 height: 15999)
Image box: (Origin: (0, 0) width: 8000 height: 16000)
Output image size:
(width: 8000 height: 16000)
Q

## QGIS observation: orthoimages are now self-consistent and match the aligned stereo DEM!

## Prepare raster stack

In [39]:
aligned_stereo_intersection_error_fn = aligned_stereo_dem_fn.replace("DEM","IntersectionErr")

In [40]:
outdir = "baker_csm_stack"
!mkdir -p $outdir
!cp $aligned_stereo_dem_fn $left_orthoimage_fullres_fn \
    $right_orthoimage_fullres_fn $aligned_stereo_intersection_error_fn \
    $outdir



In [41]:
!echo $aligned_stereo_dem_fn $left_orthoimage_fullres_fn \
    $right_orthoimage_fullres_fn $aligned_stereo_intersection_error_fn

try_pc_align_to_lidar_15m_maxdisp_rotationallowed/try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-DEM.tif final_ortho_left_1.0m.tif final_ortho_right_1.0m.tif try_pc_align_to_lidar_15m_maxdisp_rotationallowed/try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-IntersectionErr.tif


In [43]:
import glob
for fn in glob.glob(os.path.join(outdir,"*.tif")):
    print(fn)
    new_fn = fn.replace(".tif","_holes_filled.tif")
    !gdal_fillnodata.py -si 3 -md 500 $fn $new_fn

baker_csm_stack/final_ortho_right_1.0m.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
baker_csm_stack/mosaic_full128_USGS_LPC_WA_MtBaker_2015_*_LAS_2017_32610_first_filt_v1.3_1.0m-DEM_holes_filled.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
baker_csm_stack/try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-DEM.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
baker_csm_stack/try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-IntersectionErr.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
baker_csm_stack/final_ortho_left_1.0m.tif
0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
gd_dir = "sethv1_gdrive:resdepth-seth-all/baker20150911_csm_stereo_raster_stack"
validation_dem_fn = "/mnt/1.0_TB_VOLUME/sethv/shashank_data/VALIDATION_baker_128-1.0m-DEM.tif"
output_v68_fn = "/mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/ResDepth/torchgeo_experiments/output_baker_full_v68_11325.tif"
output_v93_fn = "/mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/ResDepth/torchgeo_experiments/output_baker_full_v93_11504.tif"
# train_val_split_geojson_fn = "
lidar_dem_mosaic_fn = "/mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/ResDepth/torchgeo_experiments/mosaic/mosaic_full128_USGS_LPC_WA_MtBaker_2015_*_LAS_2017_32610_first_filt_v1.3_1.0m-DEM_holes_filled.tif"
for fn in [
    aligned_stereo_dem_fn,
    left_orthoimage_fullres_fn,
    right_orthoimage_fullres_fn,
    aligned_stereo_intersection_error_fn,
    lidar_reference_bareground_dem,
    validation_dem_fn,
    output_v68_fn,
    output_v93_fn,
    lidar_dem_mosaic_fn
    # train_val_split_geojson_fn
]:
    print(fn)
    !rclone copy -v $fn $gd_dir

In [ ]:
!rclone ls $gd_dir

## Hole filling & redo orthoimages

In [2]:
!python /mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/ResDepth/torchgeo_experiments/hole_fill.py \
    baker_csm_stack/try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-DEM.tif \
    baker_csm_stack/try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-DEM_hole_fill_py_filled.tif

Not enough memory to perform factorization.


In [3]:
aligned_stereo_dem_fn = "baker_csm_stack/try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-DEM.tif"

In [7]:
import glob

## Mask out the validation areas / split into a new dataset
* Match the bounds of the 116+12 USGS tiles used in previous experiments for trainining+validation respectively
* Careful with nodata around the edges 


In [66]:
train_tiles_dir = "baker_csm_stack_train_112_tiles"
%mkdir -p $train_tiles_dir
val_tiles_dir = "baker_csm_stack_validation_16_tiles"
%mkdir -p $val_tiles_dir


In [54]:
import re
import glob
def get_tile_id(path, pattern="USGS_LPC_WA_MtBaker_2015_(.*)_LAS_2017"):
    m = re.search(pattern, path)
    tile_id = m.groups(0)
    return tile_id[0]


test="USGS_LPC_WA_MtBaker_2015_10UEU8397_LAS_2017_32610_first_filt_v1.3_1.0m-DEM.tif"
get_tile_id(test)

'10UEU8397'

In [ ]:

ulx, lry, lrx, uly = [586000, 5399000, 587000, 5400000]

In [73]:
# use the previous tiles from v68 and earlier experiments
old_dataset_dir = "/mnt/1.0_TB_VOLUME/sethv/shashank_data/TRAIN_tile_stack_baker_128_global_coreg"
old_val_dataset_dir = "/mnt/1.0_TB_VOLUME/sethv/shashank_data/VALIDATION_tile_stack_baker_128_global_coreg"

train_pattern = f"{old_dataset_dir}/*/lower_easton3/files_to_zip/USGS_LPC_WA_MtBaker_2015_*_LAS_2017_32610_first_filt_v1.3_1.0m-DEM.tif"
val_pattern = f"{old_val_dataset_dir}/*/lower_easton3/files_to_zip/USGS_LPC_WA_MtBaker_2015_*_LAS_2017_32610_first_filt_v1.3_1.0m-DEM.tif"
import os
def split_dataset_into_tiles_matching_old_dataset(old_dir, new_dir, dir_of_rasters_to_crop, lidar_tile_fn_pattern):
    for i, old_tile in enumerate(glob.glob(lidar_tile_fn_pattern)):
        tile_id = get_tile_id(os.path.basename(old_tile))
        bounds = rioxarray.open_rasterio(old_tile).rio.bounds()
        bounds = [round(b) for b in bounds]
        ulx, lry, lrx, uly = bounds
        print(i,tile_id,bounds)
        for fn in glob.glob(os.path.join(dir_of_rasters_to_crop,"*.tif")):
            destination = os.path.join(new_dir, tile_id, os.path.basename(fn))
            !mkdir -p $new_dir/$tile_id
            # print(destination)
            if not os.path.exists(destination):
                print(f"Creating {destination}")
                !gdal_translate -projwin $ulx $uly $lrx $lry $fn $destination

split_dataset_into_tiles_matching_old_dataset(old_dataset_dir, train_tiles_dir, outdir, train_pattern)
split_dataset_into_tiles_matching_old_dataset(old_val_dataset_dir, val_tiles_dir, outdir, val_pattern)

0 10UEU8998 [589000, 5398000, 590000, 5399000]
Creating baker_csm_stack_validation_16_tiles/10UEU8998/final_ortho_right_1.0m.tif
Input file size is 8000, 16000
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating baker_csm_stack_validation_16_tiles/10UEU8998/mosaic_full128_USGS_LPC_WA_MtBaker_2015_*_LAS_2017_32610_first_filt_v1.3_1.0m-DEM_holes_filled.tif
Input file size is 8000, 16000
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating baker_csm_stack_validation_16_tiles/10UEU8998/try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-DEM.tif
Input file size is 7929, 15895
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating baker_csm_stack_validation_16_tiles/10UEU8998/try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-IntersectionErr_holes_filled.tif
Input file size is 7929, 15895
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating baker_csm_stack_validation_16_tiles/10UEU8998/final_ortho_left_1.0m_holes_fille